In this notebook we combine the concepts from previous notebook. We pre-process tweets using our preprocessing pipeline, build embeddings and then classify them using a logistic regression model.

In [2]:
!!pip install demoji

['Collecting demoji',
 '  Downloading https://files.pythonhosted.org/packages/da/0b/d008f26ebbfd86d21117267e627f2f7359c76e5ecbeba08d8f631f4092c4/demoji-0.2.1-py2.py3-none-any.whl',
 'Requirement already satisfied: setuptools in /usr/local/lib/python3.6/dist-packages (from demoji) (49.2.0)',
 'Requirement already satisfied: requests<3.0.0 in /usr/local/lib/python3.6/dist-packages (from demoji) (2.23.0)',
 'Collecting colorama',
 '  Downloading https://files.pythonhosted.org/packages/c9/dc/45cdef1b4d119eb96316b3117e6d5708a08029992b2fee2c143c7a0a5cc5/colorama-0.4.3-py2.py3-none-any.whl',
 'Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.6/dist-packages (from requests<3.0.0->demoji) (2020.6.20)',
 'Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /usr/local/lib/python3.6/dist-packages (from requests<3.0.0->demoji) (1.24.3)',
 'Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.6/dist-packages (from requests<3.0.0->demo

In [3]:
'''
Contains helper funtions for preprocessing of twitter documents before getting their corresponding vectors
from word2vec_twitter model
Author: Anuj Gupta
'''

'''
functionality:
Discard non-english tweets
Discard Replies
Discard RTs
For now this is handle while fetch tweet text from mongo documents
process constantbrands mentions
process any other mentions
process constant brand name if any 
process hanstags
process URLs
process websites
process process_EmailIds
process 
process alphanums
'''

import re
import string
import demoji
demoji.download_codes()
from nltk.tokenize import TweetTokenizer

#gobal
PunctChars = r'''[`'“".?!,:;]'''
Punct = '%s+' % PunctChars
Entity = '&(amp|lt|gt|quot);'
printable = set(string.printable)

# helper functoins
def regex_or(*items):
	r = '|'.join(items)
	r = '(' + r + ')'
	return r

def pos_lookahead(r):
	return '(?=' + r + ')'

def neg_lookahead(r):
	return '(?!' + r + ')'

def optional(r):
	return '(%s)?' % r

def trim(transient_tweet_text):
	''' 
	trim leading and trailing spaces in the tweet text
	'''
	return transient_tweet_text.strip()

def strip_whiteSpaces(transient_tweet_text):
	'''
	Strip all white spaces
	'''
	transient_tweet_text = re.sub(r'[\s]+', ' ', transient_tweet_text)
	return transient_tweet_text

def to_LowerCase(transient_tweet_text):
	'''
	Convert tweet text to lower to lower case alphabets
	'''
	transient_tweet_text = transient_tweet_text.lower()
	return transient_tweet_text

def prune_multple_consecutive_same_char(transient_tweet_text):
	'''
	yesssssssss  is converted to yess 
	ssssssssssh is converted to ssh
	'''
	transient_tweet_text = re.sub(r'(.)\1+', r'\1\1', transient_tweet_text)
	return transient_tweet_text

def remove_spl_words(transient_tweet_text):
	transient_tweet_text = transient_tweet_text.replace('&amp;',' and ')

	return transient_tweet_text

def strip_unicode(transient_tweet_text):
    '''
    Strip all unicode characters from a tweet
    '''
    tweet = ''.join(i for i in transient_tweet_text if ord(i)<128)
    return tweet 

def process_URLs(transient_tweet_text):
	'''
	replace all URLs in the tweet text
	'''
	UrlStart1 = regex_or('https?://', r'www\.',r'bit.ly/')
	CommonTLDs = regex_or('com','co\\.uk','org','net','info','ca','biz','info','edu','in','au')
	UrlStart2 = r'[a-z0-9\.-]+?' + r'\.' + CommonTLDs + pos_lookahead(r'[/ \W\b]')
	UrlBody = r'[^ \t\r\n<>]*?'  # * not + for case of:  "go to bla.com." -- don't want period
	UrlExtraCrapBeforeEnd = '%s+?' % regex_or(PunctChars, Entity)
	UrlEnd = regex_or( r'\.\.+', r'[<>]', r'\s', '$')
	Url = 	(optional(r'\b') + 
    		regex_or(UrlStart1, UrlStart2) + 
    		UrlBody + 
    pos_lookahead( optional(UrlExtraCrapBeforeEnd) + UrlEnd))

	Url_RE = re.compile("(%s)" % Url, re.U|re.I)
	transient_tweet_text = re.sub(Url_RE, " constanturl ", transient_tweet_text)

	# fix to handle unicodes in URL
	URL_regex2 = r'\b(htt)[p\:\/]*([\\x\\u][a-z0-9]*)*'
	transient_tweet_text = re.sub(URL_regex2, " constanturl ", transient_tweet_text)
	return transient_tweet_text

def process_Websites(transient_tweet_text):
	'''
	identify website mentioned if any 
	'''
	CommonTLDs = regex_or('com','co\\.uk','org','net','info','ca','biz','info','edu','in','au')
	sep = r'[.]'
	website_regex = r'(?<!#)?(\b)?[a-zA-Z0-9.]+' + sep + CommonTLDs
	website_RE = re.compile("(%s)" % website_regex, re.U|re.I)
	transient_tweet_text = re.sub(website_RE, ' constantwebsite ', transient_tweet_text)

	return transient_tweet_text

def process_EmailIds(transient_tweet_text):
	'''
	identify email mentioned if any
	'''
	email_regex = r'(\b)?[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}(\b)?'
	transient_tweet_text = re.sub(email_regex, ' constantemailid ', transient_tweet_text)

	return transient_tweet_text

def process_Mentions(transient_tweet_text):
	'''
	Identify mentions if any
	'''
	transient_tweet_text = re.sub(r"@(\w+)", " constantnonbrandmention ", transient_tweet_text)
	return transient_tweet_text
def process_HashTags(transient_tweet_text):
	'''
	Strip all Hashtags from a tweet
	'''
	transient_tweet_text = re.sub(r"#(\w+)\b", ' constanthashtag ', transient_tweet_text)
	return transient_tweet_text

def process_Dates(transient_tweet_text):
	'''
	Identify date and convert it to constant
	'''
	#transient_tweet_text = re.sub(r'(\d+/\d+/\d+)', " constantdate " , transient_tweet_text)
	#transient_tweet_text = re.sub(r'constantnum[\s]?(/|-)[\s]?constantnum[\s]?(/|-)[\s]?constantnum', " constantdate " , transient_tweet_text)
	#date_regex = r'(constantnum)[\s]*(st|nd|rd|th)[\s]*(january|jan|february|feb|march|mar|april|may|june|jun|july|august|aug|september|sep|october|oct|november|nov|december|dec)'
	date_regex1 = r'\b((0|1|2|3)?[0-9][\s]*)[-./]([\s]*([012]?[0-9])[\s]*)([-./]([\s]*(19|20)[0-9][0-9]))?\b'
	transient_tweet_text = re.sub(date_regex1, ' constantdate ' , transient_tweet_text)
	date_regex2 = r'\b((19|20)[0-9][0-9][\s]*[-./]?)?[\s]*([012]?[0-9])[\s]*[-./][\s]*(0|1|2|3)?[0-9]\b'
	transient_tweet_text = re.sub(date_regex2, ' constantdate ' , transient_tweet_text)

	Months = regex_or('january','jan','february','feb','march','mar','april','may','june','jun','july','jul','august','aug','september','sep','october','oct','november','nov','december','dec')
	date_regex3 = r'\d+[\s]*(st|nd|rd|th)[\s]*' + Months 
	transient_tweet_text = re.sub(date_regex3, ' constantdate ', transient_tweet_text)
	date_regex4 = Months + r'[\s]*\d+[\s]*(st|nd|rd|th)*\b'
	transient_tweet_text = re.sub(date_regex4, ' constantdate ' , transient_tweet_text)
	date_regex5 = r'[\s]?\b(19|20)[0-9][0-9]\b[\s]?'
	transient_tweet_text = re.sub(date_regex5, ' constantdate ' , transient_tweet_text)

	date_regex6 = r'([\s]*(constantdate))+'
	transient_tweet_text = re.sub(date_regex6, ' constantdate ' , transient_tweet_text)

	return transient_tweet_text

def process_Times(transient_tweet_text):
	'''
	Indentify time and convert it to constant
	'''
	time_regex1 = r'([0-9]|0[0-9]|1[0-9]|2[0-3]):[0-5][0-9][\s]*(am|pm)?[\s]*([iescm](st)|gmt|utc|[pmce](dt))?'
	transient_tweet_text = re.sub(time_regex1, ' constanttime ' , transient_tweet_text)

	return transient_tweet_text

def process_BrandMentions(transient_tweet_text):
	'''
	process all airrwoot brands Mentions if any in tweet text
	'''
	constant_brands = regex_or('paytmcare', 'paytm','snapdeal_help','snapdeal','bluestone_com','shopohelp','shopo','mobikwikswat',
		'mobikwik','taxiforsure','tfscares','zoomcarindia','freshmenuindia','freshmenucares','grofers','jetairways',
		'cleartrip','olacabs','support_ola','makemytrip','makemytripcare','chai_point')

	BrandMentionRegex = r'@(\b)*' + constant_brands
	transient_tweet_text = re.sub(BrandMentionRegex, ' constantbrandmention ', transient_tweet_text)
	return transient_tweet_text

def process_NonBrandMentions(transient_tweet_text):
	'''
	process all Mentions left, if any, in tweet text
	'''
	transient_tweet_text = re.sub(r"@(\w+)", ' constantnonbrandmention ', transient_tweet_text)
	return transient_tweet_text

def process_BrandName(transient_tweet_text):
	'''
	process all airrwoot brands Mentions if any in tweet text
	'''
	constant_brands = regex_or('paytmcare', 'paytm','snapdeal_help','snapdeal','bluestone_com', 'bluestone','shopohelp','shopo',
		'mobikwikswat','mobikwik','taxiforsure','tfscares', 'tfs','zoomcarindia', 'zoomcar','freshmenuindia','freshmenucares', 
		'freshmenu','grofers','jetairways', 'jet','cleartrip','olacabs','support_ola', 'olacab', 'ola' ,'makemytrip',
		'makemytripcare', 'chai_point')

	BrandNameRegex = constant_brands
	transient_tweet_text = re.sub(BrandNameRegex, ' constantbrandname ', transient_tweet_text)
	return transient_tweet_text

def identify_Savings(transient_tweet_text):
	'''
	identify sale/save offers
	'''
	#sale_regex = r'(?<!#)\b(discount|discounts|sale|save|saver|super[\s]*saver|super[\s]*save)\b[\s]*(constantnum)*[\s]*[%]?[\s]*(-|~)?[\s]*(constantnum)*[\s]*[%]?'
	sale_regex = r'(?<!#)\b(discount|discounts|sale|save|saver|super[\s]*saver|super[\s]*save)\b[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?[\s]*(-|~|or)?[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?'
	transient_tweet_text = re.sub(sale_regex, " constantdiscount ", transient_tweet_text)
	#discount_List = []
	#discount_List = re.findall(r'constantdiscount', transient_tweet_text)
	return transient_tweet_text

def indentify_Offers(transient_tweet_text):
	'''
	identify cashbacks and off / substrings of the form "30% off" or "30% cashback" or "$30 off"
	Replace them by constantOFFER
	'''
	#transient_tweet_text = re.sub(r'[rs|$]?[ ]*[constantnum][ ]*[%]?[ ]?[off|cashback|offer]', "constantoffer", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:(up[\s]?to)?((rs|\$)*[\s]*(constantnum))[\s]*[%]?)?[\s]*[-|~|or]?[\.]?[\s]*((rs|\$)*[\s]*(constantnum))*[\s]*[%]?[\s]*(offer|off|cashback|cash|cash back)', " constantoffer ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:cashback|cash back|cash)\b', " constantoffer ", transient_tweet_text)
	#Offer_List = []
	#Offer_List = re.findall(r'constantoffer', transient_tweet_text)
	return transient_tweet_text

def indentify_Promos(transient_tweet_text):
	'''
	indentify coupons/promos with promo codes
	Assumption - promo code can be alphanumeric. But it immediately follows text of promo/code/promocode etc
	'''
	#transient_tweet_text = re.sub(r'\b(promocode|promo code|promo|code)[s]?[\s]*[a-z]*(constantnum)*[a-z]*[\s]+', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*(constantalphanum)\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*[a-z]+\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code)\b[\s]*[0-9]+\b', " constantpromo ", transient_tweet_text)
	transient_tweet_text = re.sub(r'(?<!#)\b(?:promocode|promo code|promo|coupon code|code|coupon)[s]?\b', " constantpromo ", transient_tweet_text)
	#Promo_List = []
	#Promo_List = re.findall(r'constantpromo', transient_tweet_text)
	return transient_tweet_text

def indentify_Money(transient_tweet_text):
	'''
	identify money in the tweet text but outside offers. This includes $,Rs, pound, Euro
	'''
	money_regex1 = r'\b(rs|\$)[\s]*(constantnum)?[\.]?[\s]*constantnum\b'
	transient_tweet_text = re.sub(money_regex1, " constantmoney ", transient_tweet_text)
	money_regex2 = r'[\s]*[\.]?[\s]*constantnum(cent[s]?|\$|c)\b'
	transient_tweet_text = re.sub(money_regex2, " constantmoney ", transient_tweet_text)
	money_regex3 = r'(\$|rs)[\s]*constantalphanum'
	transient_tweet_text = re.sub(money_regex3, " constantmoney ", transient_tweet_text)
	#Money_List = []
	#Money_List = re.findall(r'constantmoney', transient_tweet_text)
	return transient_tweet_text

def indentify_freebies(transient_tweet_text):
	'''
	indentify freebies in tweets if any - free offers, free shipping, free trial, 
	'''
	freebies_regex1 = r'(?<!#)\b(?:free)[\s]+[a-z]+\b'
	transient_tweet_text = re.sub(freebies_regex1, " constantfreebies ", transient_tweet_text)
	freebies_regex2 = r'(?<!#)\b(?:free)[\s]+[a-z]*\b'
	transient_tweet_text = re.sub(freebies_regex2, " constantfreebies ", transient_tweet_text)
	return transient_tweet_text

def replace_numbers(transient_tweet_text):
	'''
	Given any number/interger in tweet text, we want it to be replaced by constantnum
	'''
	# we want to process only those numbers that are not in a hashtag - below logic does this
	num_regex = r'(?<!#)\b(?:[-+]?[\d,]*[\.]?[\d,]*[\d]+|\d+)\b'
	transient_tweet_text = re.sub(num_regex, " constantnum " , transient_tweet_text)
	return transient_tweet_text

def identify_AlphaNumerics(transient_tweet_text):
	'''
	Identify alpha numerics - this helps in identifying product codes/models, promocodes, Order IDs
	'''
	AlphaNumeric_regex = r'(?<!#)\b(?:([a-z]+[0-9]+[a-z]*|[a-z]*[0-9]+[a-z]+)[a-z,0-9]*)\b'
	transient_tweet_text = re.sub(AlphaNumeric_regex, " constantalphanum ", transient_tweet_text)
	return transient_tweet_text

def strip_whiteSpaces(transient_tweet_text):
	'''
	Strip all white spaces
	'''
	transient_tweet_text = re.sub(r'[\s]+', ' ', transient_tweet_text)
	return transient_tweet_text

def prune_multple_consecutive_same_char(transient_tweet_text):
	'''
	yesssssssss  is converted to yess 
	ssssssssssh is converted to ssh
	'''
	transient_tweet_text = re.sub(r'(.)\1+', r'\1\1', transient_tweet_text)
	return transient_tweet_text

def remove_spl_words(transient_tweet_text):
	transient_tweet_text = transient_tweet_text.replace('&amp;',' and ')

	return transient_tweet_text

def remove_emoji(transient_tweet_text):
    tweet_tokenizer = TweetTokenizer()
    tokenized_tweet = tweet_tokenizer.tokenize(transient_tweet_text)
    emojis_present = demoji.findall(transient_tweet_text)
    tweet_no_emoji=''
    for i,s in enumerate(tokenized_tweet):
        if s in emojis_present.keys():
            tweet_no_emoji = tweet_no_emoji + ' ' + emojis_present[s]
        else:
            tweet_no_emoji = tweet_no_emoji + ' ' + s
    return tweet_no_emoji

def deEmojify(transient_tweet_text):
    return transient_tweet_text.encode('ascii', 'ignore').decode('ascii')

# def print_test():
    
#     test_tweet = "Nice @varun paytm @paytm saver abc@gmail.com sizes for the wolf on 20/10/2010 at 10:00PM  grey/deep royal-volt Nike Air Skylon II retro are 40% OFF for a limited time at $59.99 + FREE shipping.BUY HERE -> https://bit.ly/2L2n7rB (promotion - use code MEMDAYSV at checkout)"
#     #General Proprocessing
#     test_tweet = test_tweet.lower()
#     test_tweet = strip_unicode(test_tweet)
    
#     #function tests
#     print("Process URLS:\n",process_URLs(test_tweet))
#     print("Remove websites:\n",process_Websites(test_tweet))
#     print("Remove mentions:\n",process_Mentions(test_tweet))
#     print('Remove Emailid:\n',process_EmailIds(test_tweet))
#     print("Remove Hashtags:\n",process_HashTags(test_tweet))
#     print("Remove Dates:\n",process_Dates(test_tweet))
#     print("Process Time:\n",process_Times(test_tweet))
#     print("Process Brand Mention:\n",process_BrandMentions(test_tweet))
#     print("Process non Brand Mention:\n",process_NonBrandMentions(test_tweet))
#     print("Process Brand Name:\n",process_BrandName(test_tweet))
#     print("Process Savings:\n",identify_Savings(test_tweet))
#     print("Process Offers:\n",indentify_Offers(test_tweet))
#     print("Identiy Promos:\n",indentify_Promos(test_tweet))
# ############
# print_test()




def process_TweetText(tweet_text):
	'''
	Takes tweet_text and preprocesses it 
	Order of preprocessing:
	'''

	# get utf-8 encoding, lowercase, trim and remove multiple white spaces
	transient_tweet_text = tweet_text
	transient_tweet_text = strip_unicode(transient_tweet_text)
	#print "PROCESSED: ", transient_tweet_text

	transient_tweet_text = to_LowerCase(transient_tweet_text)
	transient_tweet_text = trim(transient_tweet_text)
	transient_tweet_text = strip_whiteSpaces(transient_tweet_text)
	transient_tweet_text = remove_spl_words(transient_tweet_text)

 
	#emoji
	transient_tweet_text = remove_emoji(transient_tweet_text) 
	transient_tweet_text = deEmojify(transient_tweet_text)
	# process Hastags, URLs, Websites, process_EmailIds
	# Give precedence to url over hashtag
	transient_tweet_text = process_URLs(transient_tweet_text)
	transient_tweet_text = process_HashTags(transient_tweet_text)
	#transient_tweet_text = process_Websites(transient_tweet_text)
	transient_tweet_text = process_EmailIds(transient_tweet_text)

	# process for brand mention, any other mention and brand Name
	#transient_tweet_text = process_BrandMentions(transient_tweet_text)
	#transient_tweet_text = process_NonBrandMentions(transient_tweet_text)
	transient_tweet_text = process_Mentions(transient_tweet_text)
	#transient_tweet_text = process_BrandName(transient_tweet_text)

	# remove any unicodes
	transient_tweet_text = strip_unicode(transient_tweet_text)

	# identify Date / Time if any
	transient_tweet_text = process_Times(transient_tweet_text)
	transient_tweet_text = process_Dates(transient_tweet_text)

	# indentify alphanums and nums
	transient_tweet_text = identify_AlphaNumerics(transient_tweet_text)
	transient_tweet_text = replace_numbers(transient_tweet_text)
	
	# identify promos, savings, offers, money and freebies
	transient_tweet_text = indentify_Promos(transient_tweet_text)
	transient_tweet_text = identify_Savings(transient_tweet_text)
	transient_tweet_text = indentify_Offers(transient_tweet_text)
	transient_tweet_text = indentify_Money(transient_tweet_text)
	transient_tweet_text = indentify_freebies(transient_tweet_text)

	transient_tweet_text = trim(transient_tweet_text)
	transient_tweet_text = strip_whiteSpaces(transient_tweet_text)

	transient_tweet_text = prune_multple_consecutive_same_char(transient_tweet_text)

	return transient_tweet_text

# if __name__ == "__main__":
# print(process_TweetText("Nice @varun paytm @paytm saver abc@gmail.com sizes for the wolf on 20/10/2010 at 10:00PM  grey/deep royal-volt Nike Air Skylon II retro are 40% OFF for a limited time at $59.99 + FREE shipping.BUY HERE -> https://bit.ly/2L2n7rB (promotion - use code MEMDAYSV at checkout)"))

... OK (Got response in 0.41 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [9]:
#Making the necessary imports
import os
import sys

#preprocessing_path = "/home/etherealenvy/Downloads/practical-nlp/Ch8/O5_smtd_preprocessing.py"
#sys.path.append(os.path.abspath(preprocessing_path))

#import O5_smtd_preprocessing

from nltk.corpus import stopwords
from string import punctuation
import nltk
nltk.download('stopwords')

import pandas as pd
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer()


#imports related to modeling
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Reading and Preprocessing
Let's read the dataset and pre-process them using the pre-processing pipeline.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [11]:

datapath = "/content/drive/My Drive/Data/NLP/Practical_NLP_Oreilly/Chap8/sts_gold_tweet.csv"
df = pd.read_csv(datapath,error_bad_lines=False,delimiter=";")
df = df.dropna(how='any')
df.drop(columns=['id'], inplace=True)
display(df.head())

#pre-process tweets using our package
tweets_no_processed = df['tweet'].values
df['tweet'] = df['tweet'].apply(lambda x: process_TweetText(x))
df['tweet_tokens'] = df['tweet'].apply(lambda x: tweet_tokenizer.tokenize(x))
df['tweet_no_stopwords'] = df['tweet_tokens'].apply(lambda x: [word for word in x if word not in stopwords.words('english')])
tweets_processed = df['tweet_tokens'].values
tweets_cat = df['polarity'].values

display(df.head())
print("Number of tweets and categories")
print(len(tweets_processed), len(tweets_cat))
print("\nExamle of polarity, processed tweet, processed tweet without stopwords")
print(tweets_cat[0],',',tweets_processed[0],',',df['tweet_no_stopwords'].values[0])

,polarity,tweet
0,0,the angel is going to miss the athlete this we...
1,0,It looks as though Shaq is getting traded to C...
2,0,@clarianne APRIL 9TH ISN'T COMING SOON ENOUGH
3,0,drinking a McDonalds coffee and not understand...
4,0,So dissapointed Taylor Swift doesnt have a Twi...


,polarity,tweet,tweet_tokens,tweet_no_stopwords
0,0,the angel is going to miss the athlete this we...,"[the, angel, is, going, to, miss, the, athlete...","[angel, going, miss, athlete, weekend]"
1,0,it looks as though shaq is getting traded to c...,"[it, looks, as, though, shaq, is, getting, tra...","[looks, though, shaq, getting, traded, clevela..."
2,0,constantnonbrandmention constantdate isn't com...,"[constantnonbrandmention, constantdate, isn't,...","[constantnonbrandmention, constantdate, coming..."
3,0,drinking a mcdonalds coffee and not understand...,"[drinking, a, mcdonalds, coffee, and, not, und...","[drinking, mcdonalds, coffee, understanding, s..."
4,0,so dissapointed taylor swift doesnt have a twi...,"[so, dissapointed, taylor, swift, doesnt, have...","[dissapointed, taylor, swift, doesnt, twitter]"


Number of tweets and categories
2034 2034

Examle of polarity, processed tweet, processed tweet without stopwords
0 , ['the', 'angel', 'is', 'going', 'to', 'miss', 'the', 'athlete', 'this', 'weekend'] , ['angel', 'going', 'miss', 'athlete', 'weekend']


## Train your own Embedding

Twitter process

In [12]:
#CBOW
import time
start = time.time()
w2v_model = Word2Vec(tweets_processed,min_count=5, sg=0)
end = time.time()

print("CBOW Model Training Complete.\nTime taken for training is:{:.5f} sec ".format((end-start)))

CBOW Model Training Complete.
Time taken for training is:0.33940 sec 


In [13]:
#Create document vectors by averaging word vectors.
def embedding_feats(list_of_lists):
    DIMENSION = 100
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        feats.append(feat_for_this/count_for_this if count_for_this > 0 else feat_for_this)        
    return feats

train_vectors = embedding_feats(df['tweet_no_stopwords'].values)
print(len(train_vectors))

2034


In [14]:
#Take any classifier (LogisticRegression here)
classifier = LogisticRegression(random_state=2020)
train_data, test_data, train_cats, test_cats = train_test_split(train_vectors, 
                                                                df['polarity'].values)

classifier.fit(train_data, train_cats)
print("Accuracy: ", classifier.score(test_data, test_cats))
preds = classifier.predict(test_data)
print(classification_report(test_cats, preds))

Accuracy:  0.6895874263261297
              precision    recall  f1-score   support

           0       0.69      1.00      0.82       351
           4       0.00      0.00      0.00       158

    accuracy                           0.69       509
   macro avg       0.34      0.50      0.41       509
weighted avg       0.48      0.69      0.56       509



Twitter no process

In [16]:
datapath = "Path to repo"
df = pd.read_csv(datapath+"/practical-nlp/Ch8/Data/sts_gold_tweet.csv",
                 error_bad_lines=False,delimiter=";")
df = df.dropna(how='any')
df.drop(columns=['id'], inplace=True)
display(df.head())

FileNotFoundError: ignored

In [ ]:
#CBOW
import time
start = time.time()
w2v_model = Word2Vec(tweets_processed,min_count=5, sg=0)
end = time.time()

print("CBOW Model Training Complete.\nTime taken for training is:{:.5f} sec ".format((end-start)))

In [ ]:
#Create document vectors by averaging word vectors.
def embedding_feats(list_of_lists):
    DIMENSION = 100
    zero_vector = np.zeros(DIMENSION)
    feats = []
    for tokens in list_of_lists:
        feat_for_this =  np.zeros(DIMENSION)
        count_for_this = 0
        for token in tokens:
            if token in w2v_model:
                feat_for_this += w2v_model[token]
                count_for_this +=1
        feats.append(feat_for_this/count_for_this if count_for_this > 0 else feat_for_this)        
    return feats

train_vectors = embedding_feats(df['tweet_no_stopwords'].values)
print(len(train_vectors))

In [ ]:
#Take any classifier (LogisticRegression here)
classifier = LogisticRegression(random_state=2020)
train_data, test_data, train_cats, test_cats = train_test_split(train_vectors, 
                                                                df['polarity'].values)

classifier.fit(train_data, train_cats)
print("Accuracy: ", classifier.score(test_data, test_cats))
preds = classifier.predict(test_data)
print(classification_report(test_cats, preds))